# Segmenting and Clustering Neighborhoods in Toronto - II
### Exercise 2

## Collect detils from web 

Obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [1]:
import pandas as pd

This logic is descibed in the previous notebook in detail.
* <a href="Segmenting and Clustering Neighborhoods in Toronto.ipynb">Previous Notebook</a>

It bring the data from the wikipedia page and the postal code from TORONTO. 

In [2]:
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url)[0] #pd.read_html(url, skiprows=1)[0]
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()

df = df[df['Borough'] != 'Not assigned']

df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

def row_handler(row):
    if row['Neighbourhood'] == 'Not assigned':
        return row['Borough'] 
    return row['Neighbourhood'] 

df['Neighbourhood'] = df.apply(row_handler, axis=1)

import numpy as np
df['Latitude'] = np.nan
df['Longitude'] = np.nan

df.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",NaN,NaN
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN


#### My Struggle and the break-through
I struggled a lot with geocoder and I was using the google. The google map was failing all the time and the while loop go infinite. Then I realized that the issue is with the google api.
<br>
I then switched to <b> ArcGIS</b> provider and then it worked

<hr>
#### Tried the following methods but didnt work-out
<pre>

import requests # library to handle requests
geo_url = "https://geocoder.ca/?locate=101%20Wellington%20St,%20Ottawa&geoit=xml&json=1"
results = requests.get(url)
</pre>

In [3]:
#!conda install -c conda-forge geocoder

In [4]:
# This code was my breakthrough
import geocoder
g = geocoder.arcgis('Mountain View, CA')
print(g.latlng)


[37.389670000000024, -122.08159999999998]


In [5]:

import geocoder # import geocoder


for index, row in df.iterrows():
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(row['Postcode']))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    df.at[index, 'Latitude'] = latitude
    df.at[index, 'Longitude'] = longitude
    print(longitude,latitude, row['Postcode'] )


-79.19556138899998 43.81165000000004 M1B
-79.15870110299994 43.78560500000003 M1C
-79.17529924399997 43.76569000000006 M1E
-79.21760999999998 43.76821641200007 M1G
-79.23943999999995 43.769608066000046 M1H
-79.23217158499995 43.743085000000065 M1J
-79.26366999999993 43.72625965800006 M1K
-79.28490999999997 43.71321309600006 M1L
-79.23497617799995 43.72357500000004 M1M
-79.26006895899997 43.696690000000046 M1N
-79.26897402899993 43.759975000000054 M1P
-79.30055999999996 43.75080267200008 M1R
-79.26797613999997 43.79394000000008 M1S
-79.29924441799994 43.78472500000004 M1T
-79.28014659999997 43.81759500000004 M1V
-79.32073999999994 43.80069800200005 M1W
-79.21670085099998 43.83421500000003 M1X
-79.35623615099996 43.80284500000005 M2H
-79.34778216099994 43.78081000000003 M2J
-79.38054242199996 43.781015000000025 M2K
-79.37993999999998 43.75717700000007 M2L
-79.41354639199994 43.791300000000035 M2M
-79.40741984599998 43.76816500000007 M2N
-79.40006199699997 43.74781000000007 M2P
-79.445699

In [6]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440


As I have got the pandas frame ready, I save this in a picke file so that I can get it later as-is, in order to reduce the call and to save the time 

In [7]:
df.to_pickle("./dummy.pkl")

In [8]:
df = pd.read_pickle("./dummy.pkl")

In [9]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')

Folium installed


This is old commented logic: donnt work
<!--
geolocator = Nominatim(user_agent="coursera-course.com")
location =  None
for index, row in df.iterrows():

    while(location is None):
        try:
            location = geolocator.geocode(row['Postcode'])     
        except:
            
    row['Latitude'] = location.latitude
    row['Longitude'] = location.longitude
    print(row['Postcode'], row['Latitude'] ,  row['Longitude'])
    location = None    
df.head()
-->

<!--
import geocoder # import geocoder

for index, row in df.iterrows():

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(row['Postcode']))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    df.at[index, 'Latitude'] = latitude
    df.at[index, 'Longitude'] = longitude
    
df.head()
-->

In [10]:
df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [11]:
import geocoder
g = geocoder.arcgis('Toronto, ON')

#print(g.latlng)
#address = 'Toronto, ON'
#geolocator = Nominatim()
#location = geolocator.geocode(address)

latitude = g.latlng[0]
longitude = g.latlng[1]
print(latitude, longitude)

43.648690000000045 -79.38543999999996


In [12]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

map_address = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_address)  
    
map_address

Thanks a lot for visiting this notebook... <b>have a great day
    <hr>
    <b>Sharafudheen